In [1]:
import mechanism
import pandas as pd
import networkx as nx
#from mbi.junction_tree import JunctionTree
#from mbi import Domain
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import spectral_clustering
import itertools
import json
from operator import itemgetter

In [2]:
# find the best queries to ask by non-privately analyzing the provisional dataset (assumed to be public)
my_domain = json.load(open('domain.json'))
data = pd.read_csv('competitor_pack/data/fire-data.csv', usecols = my_domain["columns"])
specs = json.load(open('competitor_pack/data/fire-data-specs.json'))
mapping = json.load(open('competitor_pack/data/fire-data-specs-mapping.json'))

data.head()

,ALS Unit,Final Priority,Call Type Group,Original Priority,Priority,City,Unit Type,Fire Prevention District,Battalion,Supervisor District,...,Hospital DtTm,Location - Lng,Number of Alarms,Available DtTm,Unit sequence in call dispatch,Location - Lat,Call Date,Unit ID,Box,Address
0,0,0,0,0,0,0,0,0,0,0,...,NaN,-122.426379,1,1.451604e+09,2,37.776688,1451602800,432,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1.451606e+09,-122.426379,1,1.451607e+09,1,37.776688,1451602800,418,0,0
2,1,0,1,0,0,0,1,1,1,1,...,NaN,-122.394748,1,1.451603e+09,1,37.794480,1451602800,245,1,1
3,1,1,2,1,1,0,2,2,2,2,...,NaN,-122.409572,1,1.451604e+09,1,37.747553,1451602800,87,2,2
4,0,1,3,1,1,0,3,3,3,3,...,NaN,-122.488371,1,1.451604e+09,3,37.731150,1451602800,97,3,3


In [3]:
M = mechanism.Mechanism(data, specs, my_domain)

#my_data = M.transform_domain(my_data, mapping)
#my_data.to_csv(data)

#M = mechanism.Mechanism(data, specs, domain)

data = M.load_data(is_encoded = True)
domain = M.domain
for col in data:
    if data[col].isnull().all():
        del data[col]
        del domain[col]


In [4]:
data.head()

,ALS Unit,Final Priority,Call Type Group,Original Priority,Priority,City,Unit Type,Fire Prevention District,Battalion,Supervisor District,...,Hospital DtTm,Location - Lng,Number of Alarms,Available DtTm,Unit sequence in call dispatch,Location - Lat,Call Date,Unit ID,Box,Address
0,0,0,0,0,0,0,0,0,0,0,...,NaN,-122.426379,1,1.451604e+09,2,37.776688,1451602800,432,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1.451606e+09,-122.426379,1,1.451607e+09,1,37.776688,1451602800,418,0,0
2,1,0,1,0,0,0,1,1,1,1,...,NaN,-122.394748,1,1.451603e+09,1,37.794480,1451602800,245,1,1
3,1,1,2,1,1,0,2,2,2,2,...,NaN,-122.409572,1,1.451604e+09,1,37.747553,1451602800,87,2,2
4,0,1,3,1,1,0,3,3,3,3,...,NaN,-122.488371,1,1.451604e+09,3,37.731150,1451602800,97,3,3


In [5]:
def datavector(df, domain, flatten=True):
    """ return the database in vector-of-counts form """
    if type(domain) is int:
        bins = [range(domain+1)]
    else:
        bins = [range(n+1) for n in domain]
    ans = np.histogramdd(df.values, bins)[0]
    return ans.flatten() if flatten else ans


#indices = itemgetter(*['Unit ID', 'Call Type'])(domain)
#datavector(data[['Unit ID', 'Call Type']], indices)

In [6]:
# calculate mutual info between every pair of attributes

N = data.shape[0]
df = pd.DataFrame(columns=['col1','col2', 'score'])
idx = 0

for a,b in itertools.combinations(domain.keys(), 2):
    A = datavector(data[a], domain[a])/ N
    B = datavector(data[b], domain[b]) / N
    indices = itemgetter(*[a,b])(domain)
    AB = datavector(data[[a,b]], indices) / N
    nz = AB > 0
    #print(a, b, A, B, AB)

    score = np.dot(AB[nz], np.log(AB / np.outer(A,B).flatten())[nz])
    print(a,b,score)
    df.loc[idx] = [a,b,score]
    df.loc[idx+1] = [b,a,score]
    idx += 2

df = df.sort_values('score')


ALS Unit Final Priority 0.008415982492928756
ALS Unit Call Type Group 0.06392677133697044
ALS Unit Original Priority 0.011205773474697579
ALS Unit Priority 0.011114866139575309
ALS Unit City 0.0007396889954180506
ALS Unit Unit Type 0.43887410224953516
ALS Unit Fire Prevention District 0.00820374435768103
ALS Unit Battalion 0.007978732843241874
ALS Unit Supervisor District 0.008054810485372817
ALS Unit Call Final Disposition 0.0587274428546189
ALS Unit Zipcode of Incident 0.010145019958558623
ALS Unit Call Type 0.06701809627795918


/home/elbert/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
/home/elbert/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


ALS Unit Neighborhooods - Analysis Boundaries 0.01227568688591955
ALS Unit Station Area 0.016705933812200334
ALS Unit Watch Date 0.0
ALS Unit Received DtTm 0.0
ALS Unit Entry DtTm 0.0
ALS Unit Dispatch DtTm 0.0
ALS Unit Response DtTm 0.0
ALS Unit On Scene DtTm 0.0
ALS Unit Transport DtTm 0.0
ALS Unit Hospital DtTm 0.0
ALS Unit Location - Lng 0.0
ALS Unit Number of Alarms 0.00018829972253661022
ALS Unit Available DtTm 0.0
ALS Unit Unit sequence in call dispatch 0.023453982281068354
ALS Unit Location - Lat 0.0
ALS Unit Call Date 0.0
ALS Unit Unit ID 0.5809345754332818
ALS Unit Box 0.02948984323526331
ALS Unit Address 0.06908063703371009
Final Priority Call Type Group 0.2962250004252005
Final Priority Original Priority 0.49097853511003714
Final Priority Priority 0.6061757750410857
Final Priority City 0.00014841481246612918
Final Priority Unit Type 0.09242905567571152
Final Priority Fire Prevention District 0.0006653459326759975
Final Priority Battalion 0.0005923918984668559
Final Priority

/home/elbert/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app


City Received DtTm 0.0
City Entry DtTm 0.0
City Dispatch DtTm 0.0
City Response DtTm 0.0
City On Scene DtTm 0.0
City Transport DtTm 0.0
City Hospital DtTm 0.0
City Location - Lng 0.0
City Number of Alarms 2.7296721788081426e-05
City Available DtTm 0.0
City Unit sequence in call dispatch 0.0017507143058907264
City Location - Lat 0.0
City Call Date 0.0
City Unit ID 0.03825038652640883
City Box 0.0858914724885339
City Address 0.08993065943799759
Unit Type Fire Prevention District 0.01579085884412573
Unit Type Battalion 0.01684217239793116
Unit Type Supervisor District 0.017769844683184037
Unit Type Call Final Disposition 0.21122686230987475
Unit Type Zipcode of Incident 0.022582600338376432
Unit Type Call Type 0.25454771994349756
Unit Type Neighborhooods - Analysis Boundaries 0.02449602509581566
Unit Type Station Area 0.02360006434326553
Unit Type Watch Date 0.0
Unit Type Received DtTm 0.0
Unit Type Entry DtTm 0.0
Unit Type Dispatch DtTm 0.0
Unit Type Response DtTm 0.0
Unit Type On Scene 

Station Area Location - Lng 0.0
Station Area Number of Alarms 0.004298912167407068
Station Area Available DtTm 0.0
Station Area Unit sequence in call dispatch 0.013936619806516655
Station Area Location - Lat 0.0
Station Area Call Date 0.0
Station Area Unit ID 1.2520311833232274
Station Area Box 3.439971806180039
Station Area Address 3.410614842921552
Watch Date Received DtTm 0.0
Watch Date Entry DtTm 0.0
Watch Date Dispatch DtTm 0.0
Watch Date Response DtTm 0.0
Watch Date On Scene DtTm 0.0
Watch Date Transport DtTm 0.0
Watch Date Hospital DtTm 0.0
Watch Date Location - Lng 0.0
Watch Date Number of Alarms 0.0
Watch Date Available DtTm 0.0
Watch Date Unit sequence in call dispatch 0.0
Watch Date Location - Lat 0.0
Watch Date Call Date 0.0
Watch Date Unit ID 0.0
Watch Date Box 0.0
Watch Date Address 0.0
Received DtTm Entry DtTm 0.0
Received DtTm Dispatch DtTm 0.0
Received DtTm Response DtTm 0.0
Received DtTm On Scene DtTm 0.0
Received DtTm Transport DtTm 0.0
Received DtTm Hospital DtTm 0.

In [7]:
#dom2 = Domain.fromdict(dict(data.df.nunique()))
nodes = domain.keys()
weights = df[df.col1.isin(nodes) & df.col2.isin(nodes)]
#weights.loc[(df.col1 == 'SEX') & (df.col2 == 'CITY'), 'score'] += 100
#weights.loc[(df.col1 == 'SEX') & (df.col2 == 'INCWAGE'), 'score'] += 100
#weights.loc[(df.col1 == 'CITY') & (df.col2 == 'INCWAGE'), 'score'] += 100
#weights.loc[(df.col1 == 'CITY') & (df.col2 == 'SEX'), 'score'] += 100
#weights.loc[(df.col1 == 'INCWAGE') & (df.col2 == 'SEX'), 'score'] += 100
#weights.loc[(df.col1 == 'INCWAGE') & (df.col2 == 'CITY'), 'score'] += 100

G = nx.Graph()
G.add_nodes_from(nodes)

for e1, e2, w in zip(weights['col1'], weights['col2'], weights['score']):
    G.add_edge(e1, e2, weight=w)
    
mst = nx.maximum_spanning_tree(G)
print(list(mst.edges))
print(len(list(mst.edges)))

[('ALS Unit', 'Unit ID'), ('ALS Unit', 'Watch Date'), ('ALS Unit', 'Received DtTm'), ('ALS Unit', 'Entry DtTm'), ('ALS Unit', 'Dispatch DtTm'), ('ALS Unit', 'Response DtTm'), ('ALS Unit', 'On Scene DtTm'), ('ALS Unit', 'Transport DtTm'), ('ALS Unit', 'Hospital DtTm'), ('ALS Unit', 'Location - Lng'), ('ALS Unit', 'Available DtTm'), ('ALS Unit', 'Location - Lat'), ('ALS Unit', 'Call Date'), ('Final Priority', 'Priority'), ('Call Type Group', 'Call Type'), ('Call Type Group', 'Priority'), ('Original Priority', 'Priority'), ('City', 'Address'), ('Unit Type', 'Unit ID'), ('Fire Prevention District', 'Address'), ('Battalion', 'Box'), ('Supervisor District', 'Address'), ('Call Final Disposition', 'Address'), ('Zipcode of Incident', 'Address'), ('Call Type', 'Address'), ('Neighborhooods - Analysis Boundaries', 'Address'), ('Station Area', 'Box'), ('Number of Alarms', 'Address'), ('Unit sequence in call dispatch', 'Address'), ('Unit ID', 'Address'), ('Box', 'Address')]
31


In [8]:
#a = 'Unit ID'
#b = 'Address'
#c = 'Unit Type'
#indices1 = itemgetter(*[a,b])(domain)
#AB = datavector(data[[a,b]], indices1, flatten = False) / N
#indices2 = itemgetter(*[a,c])(domain)
#AC = datavector(data[[a,c]], indices2, flatten = False) / N
#print(AB.shape, AC.shape)
#print(AB.transpose().shape)

In [14]:
res = pd.DataFrame(columns=['A','B','C','size','score'])
i = 0
N = data.shape[0]
print(N)
for a in nodes:
    for b, c in itertools.combinations(mst.neighbors(a), 2):
        indices1 = itemgetter(*[a,b])(domain)
        AB = datavector(data[[a,b]], indices1, flatten = False) / N
        indices2 = itemgetter(*[a,c])(domain)
        AC = datavector(data[[a,c]], indices2, flatten = False) / N
        A, B = AB.shape
        A, C = AC.shape
        print(A,B,C)
        #print(AB,AC)
        
        AB, AC = AB.reshape(A,B,1), AC.reshape(A,1,C)
        
        if A*B*C <= 10e6:
        
            ABC2 = AB * AC / AB.sum(axis=1, keepdims=True)
            indices = itemgetter(*[a,b,c])(domain)
            ABC = datavector(data[[a,b,c]], indices, flatten = False) / N
            #a3 = ABC.shape[0]
            #print(a3)
            #ABC = np.pad(ABC, [(0,a-a3),(0,0)], mode = "constant", constant_values = (0))
            #ABC = ABC.reshape(a,b,c)

            score = np.nansum(np.abs(ABC - ABC2))
            #AB = data.project([A,B]).datavector()
            #AC = data.project([A,C]).datavector()
            print(a,b,c, score)
            res.loc[i] = [a,b,c,A*B*C,score]
            i = i + 1
        else:
            print(a,b,c, "Too large")

res[res.score >= 0.05].sort_values('size',ascending=True)

305133
2 742 1000
ALS Unit Unit ID Watch Date 0.0
2 742 1000
ALS Unit Unit ID Received DtTm 0.0
2 742 1000
ALS Unit Unit ID Entry DtTm 0.0
2 742 1000
ALS Unit Unit ID Dispatch DtTm 0.0
2 742 1000
ALS Unit Unit ID Response DtTm 0.0
2 742 1000
ALS Unit Unit ID On Scene DtTm 0.0
2 742 1000
ALS Unit Unit ID Transport DtTm 0.0
2 742 1000
ALS Unit Unit ID Hospital DtTm 0.0
2 742 1000
ALS Unit Unit ID Location - Lng 0.0
2 742 1000
ALS Unit Unit ID Available DtTm 0.0
2 742 1000
ALS Unit Unit ID Location - Lat 1.2397216567661302e-16
2 742 1000
ALS Unit Unit ID Call Date 0.0
2 1000 1000
ALS Unit Watch Date Received DtTm 0.0


/home/elbert/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


2 1000 1000
ALS Unit Watch Date Entry DtTm 0.0
2 1000 1000
ALS Unit Watch Date Dispatch DtTm 0.0
2 1000 1000
ALS Unit Watch Date Response DtTm 0.0
2 1000 1000
ALS Unit Watch Date On Scene DtTm 0.0
2 1000 1000
ALS Unit Watch Date Transport DtTm 0.0
2 1000 1000
ALS Unit Watch Date Hospital DtTm 0.0
2 1000 1000
ALS Unit Watch Date Location - Lng 0.0
2 1000 1000
ALS Unit Watch Date Available DtTm 0.0
2 1000 1000
ALS Unit Watch Date Location - Lat 0.0
2 1000 1000
ALS Unit Watch Date Call Date 0.0
2 1000 1000
ALS Unit Received DtTm Entry DtTm 0.0
2 1000 1000
ALS Unit Received DtTm Dispatch DtTm 0.0
2 1000 1000
ALS Unit Received DtTm Response DtTm 0.0
2 1000 1000
ALS Unit Received DtTm On Scene DtTm 0.0
2 1000 1000
ALS Unit Received DtTm Transport DtTm 0.0
2 1000 1000
ALS Unit Received DtTm Hospital DtTm 0.0
2 1000 1000
ALS Unit Received DtTm Location - Lng 0.0
2 1000 1000
ALS Unit Received DtTm Available DtTm 0.0
2 1000 1000
ALS Unit Received DtTm Location - Lat 0.0
2 1000 1000
ALS Unit Rece

/home/elbert/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide


Unit ID Unit Type ALS Unit 7.75755124742851e-18
2089 17704 46
Box Address Station Area Too large
2089 17704 11
Box Address Battalion Too large
2089 46 11
Box Station Area Battalion 0.0004027631312110366
17704 2089 42
Address Box Neighborhooods - Analysis Boundaries Too large
17704 2089 28
Address Box Zipcode of Incident Too large
17704 2089 11
Address Box Fire Prevention District Too large
17704 2089 742
Address Box Unit ID Too large
17704 2089 12
Address Box Supervisor District Too large
17704 2089 32
Address Box Call Type Too large
17704 2089 15
Address Box Call Final Disposition Too large
17704 2089 83
Address Box Unit sequence in call dispatch Too large
17704 2089 9
Address Box City Too large
17704 2089 5
Address Box Number of Alarms Too large
17704 42 28
Address Neighborhooods - Analysis Boundaries Zipcode of Incident Too large
17704 42 11
Address Neighborhooods - Analysis Boundaries Fire Prevention District 0.0045673296667772265
17704 42 742
Address Neighborhooods - Analysis Boun

,A,B,C,size,score
82,Call Type,Call Type Group,Address,2832640,0.262132
103,Address,Call Type,Call Final Disposition,8497920,0.533082


In [15]:
np.nansum(AB*AC/AB.sum(axis=1, keepdims=True))

1.0000000000000002

In [16]:
print([tuple(row) for row in res.loc[res.score >= 0.01,['A','B','C']].values])

[('Call Type Group', 'Call Type', 'Priority'), ('Priority', 'Original Priority', 'Call Type Group'), ('Call Type', 'Call Type Group', 'Address'), ('Address', 'Neighborhooods - Analysis Boundaries', 'Supervisor District'), ('Address', 'Zipcode of Incident', 'Supervisor District'), ('Address', 'Zipcode of Incident', 'Call Final Disposition'), ('Address', 'Supervisor District', 'Call Type'), ('Address', 'Supervisor District', 'Call Final Disposition'), ('Address', 'Call Type', 'Call Final Disposition')]


In [23]:
ans = []
for col in nodes:
    lookup = res.loc[res.A == col]
    G = nx.Graph()
    G.add_nodes_from(mst.neighbors(col))

    for e1, e2, w in zip(lookup['B'], lookup['C'], lookup['score']):
        if w >= 0.001:
            G.add_edge(e1, e2, weight=w)
    #print(col, G.nodes, G.edges)
    tree = nx.maximum_spanning_tree(G)
    #print(tree.edges)
    ans += list(tree.edges)
    for e in tree.edges:
        print(e)
    ans += [(col,) + e for e in tree.edges]
print(ans)
which_used = {}
for i in ans:
    for col in i:
        which_used[col] = 1
print(which_used)

('Call Type', 'Priority')
('Original Priority', 'Call Type Group')
('Call Type Group', 'Address')
('Neighborhooods - Analysis Boundaries', 'Supervisor District')
('Zipcode of Incident', 'Supervisor District')
('Zipcode of Incident', 'Call Final Disposition')
('Fire Prevention District', 'Call Final Disposition')
('Call Type', 'Call Final Disposition')
('Call Type', 'Number of Alarms')
('Call Final Disposition', 'City')
('Unit sequence in call dispatch', 'Number of Alarms')
[('Call Type', 'Priority'), ('Call Type Group', 'Call Type', 'Priority'), ('Original Priority', 'Call Type Group'), ('Priority', 'Original Priority', 'Call Type Group'), ('Call Type Group', 'Address'), ('Call Type', 'Call Type Group', 'Address'), ('Neighborhooods - Analysis Boundaries', 'Supervisor District'), ('Zipcode of Incident', 'Supervisor District'), ('Zipcode of Incident', 'Call Final Disposition'), ('Fire Prevention District', 'Call Final Disposition'), ('Call Type', 'Call Final Disposition'), ('Call Type', 

In [19]:
with open('queries.txt', 'w') as f:
    for item in ans:
        f.write('_'.join(str(s) for s in item) + '\n')

In [28]:
domain = json.load(open("domain.json"))
domain["columns"] = list(which_used.keys())
json.dump(domain, open('domain.json', 'w'))

In [27]:
list(which_used.keys())

['Call Type',
 'Priority',
 'Call Type Group',
 'Original Priority',
 'Address',
 'Neighborhooods - Analysis Boundaries',
 'Supervisor District',
 'Zipcode of Incident',
 'Call Final Disposition',
 'Fire Prevention District',
 'Number of Alarms',
 'City',
 'Unit sequence in call dispatch']